In [8]:
import pandas as pd
import numpy as np
import unittest
import logging
import os.path
from pandas.util.testing import assert_frame_equal
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

class panda_reader():
    filename = None
    worksheetname = None
    df = None

    # Getters and setters for filename, worksheetname, and df
    def getFilename(self):
        if self.filename is None:
            logger.info('filename is not yet set!!')
            return None
        else:
            logger.debug('About to return current filename, which is: ' + self.filename)
        
        return self.filename
        
    # Setter for filename. Returns True if the file was found, False otherwise. 
    # Results can be disregarded if you are writing the file. 
    def setFilename(self, fn):
        logger.debug('About to set filename to: ' + fn)
        self.filename = fn
        if not os.path.isfile(fn):
            logger.warning('Cannot find file ' + fn)
            return False
        return True

        
    def getWorksheetName(self):
        return self.worksheetname
    
    def setWorksheetName(self, wks):
        logger.debug('About to set worksheet to: ' + wks)
        self.worksheetname = wks
        
    def getDf(self):
        return self.df

    # Return a tiny test dataframe
    def test_df(self):
        data = np.array([['','Col1','Col2'],
                         ['Row1',1,2],
                         ['Row2',3,4]])
                
        return (pd.DataFrame(data=data[1:,1:], index=data[1:,0], columns=data[0,1:]))
    
    # Write the given df to the excelFileName and excelWorksheet (unless they have already been 
    # provided and are then optional.)
    def write_df_to_excel(self, df=None, excelFileName=None, excelWorksheet=None):
        # Use df (passed param) if it's set, self.df otherwise.
        if df is None:
            mydf = self.df
        else:
            mydf = df
        fn  = excelFileName or self.filename # If excelFileName is set, then use it. If None, then use self.filename.
        wks = excelWorksheet or self.worksheetname # Use the passed excelWorksheet, if available. 
        self.setFilename(fn)
        self.setWorksheetName(wks)
        writer = pd.ExcelWriter(fn)
        mydf.to_excel(writer, wks)
        writer.save()
        
    # Read the given excelFileName and excelWorksheet (unless they have already been 
    # provided and are then optional.) return the df in the given worksheet. 
    def read_df_from_excel(self, excelFileName=None, excelWorksheet=None):
        logger.debug("Starting read_df_from_excel")
        fn = excelFileName or self.filename
        fileFound = self.setFilename(fn)
        logger.debug("Will read from the Excel file: " + fn)
        if fileFound:
            wks = excelWorksheet or self.worksheetname
            self.setWorksheetName(excelWorksheet)
            logger.debug("Will read from the worksheet: " + wks)
            self.df = pd.read_excel(fn, sheet_name=wks)

            return self.df
        else:
            logger.error("Can't find Excel file: " + fn)
            return None

class Test_panda_reader(unittest.TestCase):
    # tempExcel = '/Users/Taylor/pythonLibrary/test.xlsx' # For Taylor's testing
    tempExcel = './test.xlsx' # This will work for Rajah's testing. 
    tempWorksheet = 'testWks'
    def testFilename(self):
        logger.info('starting testFilename.')
        testPr = panda_reader()
        testFilename = 'Fred.xlsx'
        testPr.setFilename(testFilename)
        self.assertEqual(testPr.getFilename(), testFilename)
        logger.info('exiting testFilename')
    
    def testWorksheetName(self):
        logger.info('starting testWorksheetName.')
        testPr = panda_reader()
        testWorksheet = 'First sheet'
        testPr.setWorksheetName(testWorksheet)
        self.assertEqual(testPr.getWorksheetName(), testWorksheet)
        logger.info('exiting testWorksheetName')
        
    def test_test_df(self):
        logger.info('starting test_test_df.')
        testPr = panda_reader()
        df = testPr.test_df()
        logger.debug('DF contains:' + df.head())
        logger.info('exiting test_test_df')
        
    def test_write_df(self):
        logger.info('Starting test_write_df.')
        testPr = panda_reader()
        tinydf = testPr.test_df()
        testPr.setFilename(self.tempExcel)
        testPr.setWorksheetName(self.tempWorksheet)
        testPr.write_df_to_excel(tinydf)
        logger.info('Ending test_write_df.')
        
    def test_read_df_missing_file(self):
        logger.info('Starting test_read_df_missing_file.')
        myTestExcel = 'testnasiugq.xlsx' #does not exsist
        myTestWorksheet = 'Sheet7'
        testPr = panda_reader()
        mydf = testPr.read_df_from_excel(myTestExcel, myTestWorksheet)
        self.assertEqual(testPr.getFilename(), myTestExcel)
        self.assertIsNone(mydf)
        logger.info('Ending test_read_df_missing_file.')
    
    def test_read_df(self):
        logger.info('Starting test_read_df.')
        testPr = panda_reader()
        mydf = testPr.read_df_from_excel(self.tempExcel, self.tempWorksheet)
        self.assertEqual(testPr.getWorksheetName(), self.tempWorksheet)
        testdf = testPr.test_df()
        testdf['Col1'] = testdf['Col1'].astype(np.int64)
        testdf['Col2'] = testdf['Col2'].astype(np.int64)
        assert_frame_equal(mydf, testdf)
        logger.debug(mydf.head())
        logger.info('Ending test_read_df.')
        
    
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

INFO:__main__:starting testFilename.
DEBUG:__main__:About to set filename to: Fred.xlsx
DEBUG:__main__:About to return current filename, which is: Fred.xlsx
INFO:__main__:exiting testFilename
.INFO:__main__:starting testWorksheetName.
DEBUG:__main__:About to set worksheet to: First sheet
INFO:__main__:exiting testWorksheetName
.INFO:__main__:Starting test_read_df.
DEBUG:__main__:Starting read_df_from_excel
DEBUG:__main__:About to set filename to: ./test.xlsx
DEBUG:__main__:Will read from the Excel file: ./test.xlsx
DEBUG:__main__:About to set worksheet to: testWks
DEBUG:__main__:Will read from the worksheet: testWks
DEBUG:__main__:      Col1  Col2
Row1     1     2
Row2     3     4
INFO:__main__:Ending test_read_df.
.INFO:__main__:Starting test_read_df_missing_file.
DEBUG:__main__:Starting read_df_from_excel
DEBUG:__main__:About to set filename to: testnasiugq.xlsx
DEBUG:__main__:Will read from the Excel file: testnasiugq.xlsx
ERROR:__main__:Can't find Excel file: testnasiugq.xlsx
DEBUG